In [1]:
!pip install gensim konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh

     |████████████████████████████████| 19.4 MB 6.3 MB/s 
     |████████████████████████████████| 448 kB 70.6 MB/s 
     |████████████████████████████████| 86 kB 8.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-12-13 02:50:44--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP reque

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.cuda as cuda
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from google.colab import drive
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')

drive.mount('/content/drive/')

Training MNIST Model on cuda
Mounted at /content/drive/


In [19]:
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_sr_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_rs_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_ri_train_1210.csv")
df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_rd_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_all_train_1210.csv")
#df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_aug_X_train_1210.csv")
df

,Unnamed: 0,content,label
0,1418,황야의 마녀 마지막까지 발암이라 보면서 많이 답답했지만 그거 빼면 최고,1
1,1103,결말빼고는 아드레날린팍팍!! 남자들 강추임,1
2,624,영화 스토리도 탄탄하고..덴젤도 멋있고..짱짱!,0
3,318,난 앨리가 스타 포기하고 돌아와서 노래부르면서 다니는 행복한 결말만 생각하고있었는데...,1
4,467,40년전 영화인데 지금 봐도 개 쩌네 특히나 후반부 액션씬 쩐다 야외에서 소룡이 성...,1
...,...,...,...
3097,27,알파치노에 의한 영화.음악에 맞춰 아름다운 여인과 화상적으로 추는 모습은 인상적이다...,1
3098,1414,아녜스의 마지막 공연 '춘희'의 감동이 아직도 사라지지 않아요 발레의 아름다움을 한...,1
3099,382,영화사에서 최고의 엔딩 씬이 아닐까.. 웃기면서 마지막에 울리는,1
3100,342,지루하지만 꽤 재밌었다,0


## For Down Sampling



In [20]:
# If needed down sampling
np.random.seed(42)

df_0 = df[df['label'] == 0]
df_1 = df[df['label'] == 1]

min_len = min(len(df_0), len(df_1))

df = pd.concat([df_0[:min_len], df_1[:min_len]])
df = df.sample(frac=1, random_state=42)

train_df = df
test_df = pd.read_csv("/content/drive/Shareddrives/실전기계_project/data/data_spo_test_1211.csv")

TRAIN_SET_LENGTH = len(train_df)
TEST_SET_LENGTH = len(test_df)

train_df

,Unnamed: 0,content,label
1139,509,"극장판 엔딩, 감독판 엔딩. 어느하나를 꼽는게 아닌,두 엔딩이 함께 존재함으로서 이...",1
895,199,재밌나요? 더빙을 보는데 ㅠ자막이더좋긴한데,0
536,1461,따뜻한 영화네요. 실제 제 상황이랑 비슷해서 눈물이..이 영화의 행복한 결말이 정말...,1
1772,58,중반 이후로 흥미진진했다. 마지막까지 될지 몰라서 '제발 성공해라.. 제발!!' 이...,1
1111,982,통속적이지만 미국적 느와르의 정석과 연출이,0
...,...,...,...
3083,554,매회 살인자가 나오지만 가족의 따뜻함을 물씬 느끼게 해주는 수사드라마,1
2128,871,"소년병의 삶에 대한 이해, 반군에 대한 새로운 시각 또는 편견무엇이 진실인지는 모르...",0
2204,104,재밋네요 달팽이가 빨라서 더 재밌었어요,0
2546,609,피아니스트와 같은 동급 영화라생각합니다 보시면 후회없을거예요 실화영화이니요,0


In [21]:
document = []
mecab = Mecab()

# document tokenize
for doc in train_df['content']:
  document.append(mecab.morphs(doc))

In [22]:
class TextDataset(Dataset):
  def __init__(self, data, target, length, transform=None):
    super(TextDataset, self).__init__()

    self.data = data
    self.target = target
    self.length = length

  def __len__(self):
    return len(self.target)

  def __getitem__(self, idx):
    return self.data[idx], self.target[idx], self.length[idx]

## Hyperparameters

In [23]:
#BATCH_SIZES = [32]  # no arg
BATCH_SIZES = [64]  # arg
INPUT_SIZES = [32, 64]
HIDDEN_SIZES = [64, 128]
LEARNING_RATES = [1e-3, 3e-3, 5e-3]
WEIGHTDECAY = 0.9
EPOCH = 40
TRY = 3
SEED = 42

## Train

In [24]:
def train_test(BATCH_SIZE, INPUT_SIZE, HIDDEN_SIZE, LR, TRY):
  global train_df, test_df, document, WEIGHTDECAY

  tr_a, tr_l, te_a, te_l = None, None, None, None

  # word2vec train
  model = Word2Vec(document, size=INPUT_SIZE, window=5, min_count=1, sg=1, workers=2)

  def make_tensor(df):
    word_arr = [mecab.morphs(text) for text in df['content']]
    data = [torch.Tensor([model.wv[word] for word in words if word in model.wv]) for words in word_arr]
    
    # for debug when test data words not in word2vec dictionary
    #for i, d in enumerate(data):
    #  if len(d) == 0:
    #    print(df.loc[i, 'content'])
    
    doc_length = torch.LongTensor(list(map(lambda x: len(x), data)))
    target = torch.FloatTensor(df['label'].to_numpy())
    data = pad_sequence(data, batch_first=True)
      
    return data, doc_length, target

  train_data, train_doc_length, train_target = make_tensor(train_df)
  test_data, test_doc_length, test_target = make_tensor(test_df)

  train_dataloader = DataLoader(TextDataset(train_data, train_target, train_doc_length), batch_size=BATCH_SIZE, shuffle=True)
  test_dataloader = DataLoader(TextDataset(test_data, test_target, test_doc_length), batch_size=BATCH_SIZE, shuffle=True)

  class LSTM(nn.Module):
    def __init__(self):
      super(LSTM, self).__init__()
      self.lstm = nn.LSTM(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True, dropout=0.3, batch_first=True)
      self.linear_1 = nn.Linear(HIDDEN_SIZE * 2, 64)
      self.linear_2 = nn.Linear(64, 1)
      self.batchnorm_1 = nn.BatchNorm1d(64)
      self.relu = nn.ReLU()
      self.sigmoid = nn.Sigmoid()

    def forward(self, x, length):
      packed_input = pack_padded_sequence(x, length.tolist(), batch_first=True, enforce_sorted=False)
      packed_outputs, hidden = self.lstm(packed_input)
      output, output_length = pad_packed_sequence(packed_outputs, batch_first=True)
      last_seq_idxs = torch.LongTensor([x - 1 for x in output_length])
      output = output[range(output.shape[0]), last_seq_idxs, :]
      x = self.batchnorm_1(self.relu(self.linear_1(output)))
      x = self.sigmoid(self.linear_2(x))
      return x.view(-1)

  import os
  os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

  model = LSTM()
  model.to(device)

  def weight_init_kaiming_normal(submodule):
    if isinstance(submodule, torch.nn.RNN) or isinstance(submodule, torch.nn.LSTM):
      for name, param in submodule.named_parameters():
        if 'weight_ih' in name:
            torch.nn.init.xavier_uniform_(param.data)
        elif 'weight_hh' in name:
            torch.nn.init.orthogonal_(param.data)
        elif 'bias' in name:
            param.data.fill_(0)
    elif isinstance(submodule, torch.nn.Linear):
      torch.nn.init.kaiming_normal_(submodule.weight)
      submodule.bias.data.fill_(0.01)
    elif isinstance(submodule, torch.nn.BatchNorm1d):
      submodule.weight.data.fill_(1.0)
      submodule.bias.data.zero_()
  
  model.apply(weight_init_kaiming_normal)

  criterion = F.binary_cross_entropy
  optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHTDECAY)

  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [8, 16, 24, 32], gamma=0.5)

  # train
  for epoch in range(EPOCH):
    corrent_results_sum = 0.0
    avarage_loss = 0.0
    for i, data in enumerate(train_dataloader):
      inputs, labels, length = data
      inputs, labels, length = inputs.to(device), labels.to(device), length
      y_pred = model(inputs, length)
      loss = criterion(y_pred, labels, reduction='mean')

      y_pred_tag = torch.round(y_pred)
      corrent_results_sum += (y_pred_tag == labels).sum().float()
      avarage_loss += loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      scheduler.step()
    
    #if epoch % 10 == 0:
      #print(f"Epoch: {epoch + 1} | Acc: {corrent_results_sum * 100 / TRAIN_SET_LENGTH} | Loss: {avarage_loss / len(train_dataloader)}")
    
    if epoch == EPOCH - 1:
      tr_a, tr_l = corrent_results_sum * 100 / TRAIN_SET_LENGTH, avarage_loss / len(train_dataloader)
      print(f"Try: {TRY} | Train Acc: {corrent_results_sum * 100 / TRAIN_SET_LENGTH} | Train Loss: {avarage_loss / len(train_dataloader)}")


  # test
  corrent_results_sum = 0.0
  avarage_loss = 0.0
  for i, data in enumerate(test_dataloader):
    inputs, labels, length = data
    inputs, labels, length = inputs.to(device), labels.to(device), length
    y_pred = model(inputs, length)
    loss = criterion(y_pred, labels, reduction='mean')

    y_pred_tag = torch.round(y_pred)
    corrent_results_sum += (y_pred_tag == labels).sum().float()
    avarage_loss += loss.item()

  te_a, te_l = corrent_results_sum * 100 / TEST_SET_LENGTH, avarage_loss / len(test_dataloader)
  print(f"Try: {TRY} | Test Acc: {corrent_results_sum * 100 / TEST_SET_LENGTH} | Test Loss: {avarage_loss / len(test_dataloader)}")

  return tr_a, tr_l, te_a, te_l

In [25]:
for batch_size in BATCH_SIZES:
  for input_size in INPUT_SIZES:
    for hidden_size in HIDDEN_SIZES:
      for lr in LEARNING_RATES:
        train_acc, train_loss, test_acc, test_loss = 0.0, 0.0, 0.0, 0.0
        print(f"Case => Batch Size: {batch_size} | Input Size: {input_size} | Hidden Size: {hidden_size} | Learning Rate: {lr}")
        for t in range(1, TRY + 1):
          tr_a, tr_l, te_a, te_l = train_test(batch_size, input_size, hidden_size, lr, t)
          train_acc += tr_a
          train_loss += tr_l
          test_acc += te_a
          test_loss += te_l
        train_acc /= TRY
        train_loss /= TRY
        test_acc /= TRY
        test_loss /= TRY
        print(f"Total Average => Train Acc: {train_acc} | Train Loss: {train_loss} | Test Acc: {test_acc} | Test Loss: {test_loss}")
        print("")

Case => Batch Size: 64 | Input Size: 32 | Hidden Size: 64 | Learning Rate: 0.001
Try: 1 | Train Acc: 82.94647979736328 | Train Loss: 0.37750716598666445
Try: 1 | Test Acc: 72.2727279663086 | Test Loss: 0.6631170809268951
Try: 2 | Train Acc: 82.49516296386719 | Train Loss: 0.3897367855724023
Try: 2 | Test Acc: 72.2727279663086 | Test Loss: 0.619026243686676
Try: 3 | Train Acc: 81.49580383300781 | Train Loss: 0.39248127900824253
Try: 3 | Test Acc: 73.63636016845703 | Test Loss: 0.6113536283373833
Total Average => Train Acc: 82.31248474121094 | Train Loss: 0.3865750768557698 | Test Acc: 72.72727966308594 | Test Loss: 0.6311656509836515

Case => Batch Size: 64 | Input Size: 32 | Hidden Size: 64 | Learning Rate: 0.003
Try: 1 | Train Acc: 83.5267562866211 | Train Loss: 0.35250176063605715
Try: 1 | Test Acc: 71.36363220214844 | Test Loss: 0.664170503616333
Try: 2 | Train Acc: 83.36557006835938 | Train Loss: 0.3522653561465594
Try: 2 | Test Acc: 72.2727279663086 | Test Loss: 0.6832406967878342